In [1]:
from dataset import SiameseGoogleFer

In [2]:
import time
t = time.time()
train_path = "data/faceexp-comparison-data-train-public.csv"
test_path = "data/faceexp-comparison-data-test-public.csv"
train_dataset = SiameseGoogleFer(train_path, train_flag=True)
test_dataset = SiameseGoogleFer(test_path, train_flag=False)
print(time.time()-t)

<Response [410]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
7.227867841720581


In [3]:
print(train_dataset.__len__())
test_dataset.__len__()

78


28

In [5]:
from dataset import *

In [8]:
run train.py --epochs=1000

<Response [410]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [404]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>
<Response [410]>


ValueError: optimizer got an empty parameter list

In [3]:
acc

NameError: name 'acc' is not defined

In [ ]:
x = 